In [ ]:
# training the model

In [1]:
# generally all the modules needed
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset, random_split
import h5py
import numpy as np
import torchvision.transforms.functional as TF
import optuna
import random
from sklearn.metrics import classification_report, roc_auc_score 
from joblib import Parallel, delayed
import cv2
import glob
from optuna.importance import get_param_importances
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc
from torch.utils.data import WeightedRandomSampler
import gc

In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # If using CUDA
    torch.cuda.manual_seed_all(seed)  # If using multi-GPU
    torch.backends.cudnn.deterministic = True  # For deterministic behavior
    torch.backends.cudnn.benchmark = False     # Disable auto-tuner that introduces randomness

set_seed(42)

## Building and Training the model

The idea is to put the data into data loaders, define a flexible structured CNN, define a loss function, define a objective function, and use optuna to optimize the structure of the CNN, then use the strucuture determined by optuna and train a model with this structure. 

Due to the data being limited in the number of positive cases, the training of the model was targeted to best handle this. This included:
- Using only a training and validation set to maximize the number of positive cases the model could learn from
- a focal loss function to put emphasis on not missing positives
- using both a weighted sampler in the Dataloader and a high positive weight in the training 

In [3]:
class PatchTensorDataset(Dataset):
    def __init__(self, chunk_dir):
        self.data = []
        self.counter = 0

        paths = sorted([
            os.path.join(chunk_dir, f) 
            for f in os.listdir(chunk_dir) 
            if f.startswith('chunk') and f.endswith('.pt')
        ])
        for path in paths:
            self.data.extend(torch.load(path))
        print("initialized")

    def __len__(self):
        return len(self.data)


    def __getitem__(self, idx):
        sample = self.data[idx]
        self.counter += 1
        # if self.counter % 100 == 0:
        #     print(self.counter)
        return sample['data'], sample['label'], sample['filename'], sample['patch_id']


dataset = PatchTensorDataset("/explore/nobackup/people/cemeehan")

all_labels = [int(dataset[i][1]) for i in range(len(dataset))]

# compute class counts and weights
class_counts = np.bincount(all_labels)
class_weights = 1.0 / class_counts
sample_weights = [class_weights[label] for label in all_labels]

sampler = WeightedRandomSampler(weights = sample_weights, num_samples = len(sample_weights), replacement = True)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

train_loader = DataLoader(train_dataset, batch_size=32, sampler = sampler, shuffle = False, num_workers=2, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=32, sampler = sampler, shuffle = False, num_workers=2, pin_memory=True, persistent_workers=True)

initialized


In [4]:
class FeaturePresenceCNN(nn.Module):
    def __init__(self, num_layers=2, kernel_size=3, base_channels=16, dropout=0.0):
        super().__init__()
        layers = []
        in_channels = 6
        kernel_size = int(kernel_size)
        for i in range(num_layers):
            out_channels = base_channels * (2 ** i)
            layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=kernel_size, padding=kernel_size // 2))
            layers.append(nn.ReLU())
            layers.append(nn.MaxPool2d(2))
            if dropout > 0:
                layers.append(nn.Dropout(dropout))
            in_channels = out_channels
 
        self.features = nn.Sequential(*layers)
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(in_channels, 1)
        )
 
    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)

In [4]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1.0, gamma=2.0, pos_weight=None):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.bce = nn.BCEWithLogitsLoss(reduction='none', pos_weight=pos_weight)

    def forward(self, inputs, targets):
        bce_loss = self.bce(inputs, targets)
        pt = torch.exp(-bce_loss)
        loss = self.alpha * ((1 - pt) ** self.gamma) * bce_loss
        return loss.mean()

In [5]:
def objective(trial):
    import gc
    from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, f1_score
    import numpy as np
    import torch
    from torch import nn
    from torch.utils.data import DataLoader

    num_gpus = torch.cuda.device_count()
    gpu_id = trial.number % num_gpus if num_gpus > 0 else None
    device = torch.device(f"cuda:{gpu_id}" if num_gpus > 0 else "cpu")


    set_seed(42)

    best_val_loss = float('inf')
    best_model_state = None
    epochs_no_improve = 0
    patience = 5

    # Suggest hyperparameters
    lr = trial.suggest_float("lr", 1e-5, 1e-3, log=True)
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    base_channels = 16
    pos_weight_val = trial.suggest_float("pos_weight", 1.0, 100.0, log=True)
    num_layers = trial.suggest_int("num_layers", 1, 5)
    kernel_size = trial.suggest_int("kernel_size", 3, 5)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)

    # Model setup
    model = FeaturePresenceCNN(
        num_layers=num_layers,
        kernel_size=kernel_size,
        base_channels=base_channels,
        dropout=dropout
    ).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    pos_weight = torch.tensor([class_counts[0] / class_counts[1]]).to(device)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    for epoch in range(30):
        model.train()
        total_train_loss = 0

        for inputs, labels, filenames, patch_ids in train_loader:
            inputs = inputs.to(device)
            labels = labels.view(-1, 1).float().to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []

        with torch.no_grad():
            for inputs, labels, filenames, patch_ids in val_loader:
                inputs = inputs.to(device)
                labels = labels.view(-1, 1).float().to(device)

                outputs = model(inputs)
                val_loss += criterion(outputs, labels).item()

                all_preds.extend(torch.sigmoid(outputs).cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        avg_val_loss = val_loss / len(val_loader)
        all_probs = np.array(all_preds)
        all_labels_bin = np.array(all_labels).astype(int)

        # 🔸 Original metrics at threshold = 0.5
        all_preds_bin = (all_probs > 0.5).astype(int)
        val_acc = (all_preds_bin == all_labels_bin).mean()

        print(f"Epoch {epoch + 1} | Train loss: {total_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}")
        print(classification_report(all_labels_bin, all_preds_bin, digits=4))
        print("AUROC:", roc_auc_score(all_labels_bin, all_probs))

        # 🔸 Threshold sweep
        best_thresh = 0.5
        best_f1 = 0.0
        for thresh in np.linspace(0.1, 0.9, 17):
            preds_thresh = (all_probs > thresh).astype(int)
            recall = recall_score(all_labels_bin, preds_thresh)
            precision = precision_score(all_labels_bin, preds_thresh)
            f1 = f1_score(all_labels_bin, preds_thresh)
            print(f"Thresh={thresh:.2f} | Recall={recall:.4f} | Precision={precision:.4f} | F1={f1:.4f}")

            if f1 > best_f1:
                best_f1 = f1
                best_thresh = thresh

        print(f"📌 Best threshold by F1: {best_thresh:.2f} | Best F1: {best_f1:.4f}")

        trial.report(avg_val_loss, step=epoch)

        # Optuna pruning check
        if trial.should_prune():
            raise optuna.TrialPruned()

        # Early stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f"Stopping early after {epoch + 1} epochs (no improvement in {patience} epochs).")
                break

    # Save best model
    if best_model_state is not None:
        torch.save(best_model_state, f"model_trial_{trial.number}.pt")

    # Cleanup
    del model, optimizer, criterion, best_model_state
    torch.cuda.empty_cache()
    gc.collect()

    return best_val_loss  # ✅ still using val loss for Optuna optimization

In [6]:
sampler = optuna.samplers.TPESampler(seed=42)
study = optuna.create_study(direction="minimize",sampler=optuna.samplers.TPESampler(seed = 42))
study.optimize(objective, n_trials=15,  n_jobs=1)

# print("Best trial:")
# print(study.best_trial.params)

print("Best trial:")
print(study.best_trial.params)
print("Best value:", study.best_value)
print(f"Load model from model_trial_{study.best_trial.number}.pt")

best_trial_num = study.best_trial.number
import shutil
shutil.copyfile(f"model_trial_{best_trial_num}.pt", "final_model_2.pt")
print(f"Saved best model as final_model.pt from trial {best_trial_num}")

# Analyze:
importances = get_param_importances(study)
print(importances)

[I 2025-07-31 10:25:54,328] A new study created in memory with name: no-name-57e40686-d347-4f25-abc9-d6daf70bfa26


Epoch 1 | Train loss: 9964.8688 | Val Loss: 1.6859, Val Acc: 0.0387
              precision    recall  f1-score   support

           0     1.0000    0.0292    0.0567      3841
           1     0.0101    1.0000    0.0200        38

    accuracy                         0.0387      3879
   macro avg     0.5050    0.5146    0.0383      3879
weighted avg     0.9903    0.0387    0.0563      3879

AUROC: 0.6549692377259211
Thresh=0.10 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.15 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.20 | Recall=1.0000 | Precision=0.0098 | F1=0.0195
Thresh=0.25 | Recall=1.0000 | Precision=0.0098 | F1=0.0195
Thresh=0.30 | Recall=1.0000 | Precision=0.0099 | F1=0.0196
Thresh=0.35 | Recall=1.0000 | Precision=0.0099 | F1=0.0196
Thresh=0.40 | Recall=1.0000 | Precision=0.0100 | F1=0.0197
Thresh=0.45 | Recall=1.0000 | Precision=0.0100 | F1=0.0198
Thresh=0.50 | Recall=1.0000 | Precision=0.0101 | F1=0.0200
Thresh=0.55 | Recall=1.0000 | Precision=0.0102 |

/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

Epoch 8 | Train loss: 2696.1062 | Val Loss: 2.7785, Val Acc: 0.9835
              precision    recall  f1-score   support

           0     0.9906    0.9927    0.9917      3841
           1     0.0667    0.0526    0.0588        38

    accuracy                         0.9835      3879
   macro avg     0.5287    0.5227    0.5253      3879
weighted avg     0.9816    0.9835    0.9825      3879

AUROC: 0.6720152372600338
Thresh=0.10 | Recall=0.4211 | Precision=0.0623 | F1=0.1085
Thresh=0.15 | Recall=0.3684 | Precision=0.0686 | F1=0.1157
Thresh=0.20 | Recall=0.3421 | Precision=0.0823 | F1=0.1327
Thresh=0.25 | Recall=0.3158 | Precision=0.0923 | F1=0.1429
Thresh=0.30 | Recall=0.2895 | Precision=0.1089 | F1=0.1583
Thresh=0.35 | Recall=0.2105 | Precision=0.1039 | F1=0.1391
Thresh=0.40 | Recall=0.1316 | Precision=0.0926 | F1=0.1087
Thresh=0.45 | Recall=0.1053 | Precision=0.0909 | F1=0.0976
Thresh=0.50 | Recall=0.0526 | Precision=0.0667 | F1=0.0588
Thresh=0.55 | Recall=0.0526 | Precision=0.0833 |

[I 2025-07-31 10:34:56,433] Trial 0 finished with value: 1.2813570008846957 and parameters: {'lr': 5.6115164153345e-05, 'weight_decay': 0.006351221010640704, 'batch_size': 16, 'pos_weight': 2.0511104188433973, 'num_layers': 1, 'kernel_size': 5, 'dropout': 0.3005575058716044}. Best is trial 0 with value: 1.2813570008846957.


...

Epoch 1 | Train loss: 3343.8188 | Val Loss: 1.3416, Val Acc: 0.0098
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      3841
           1     0.0098    1.0000    0.0194        38

    accuracy                         0.0098      3879
   macro avg     0.0049    0.5000    0.0097      3879
weighted avg     0.0001    0.0098    0.0002      3879

AUROC: 0.3806917058331849
Thresh=0.10 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.15 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.20 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.25 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.30 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.35 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.40 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.45 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.50 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.55 | Recall=0.0000 | Precision=0.0000 |

/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

Epoch 2 | Train loss: 681.6895 | Val Loss: 1.3409, Val Acc: 0.0098
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      3841
           1     0.0098    1.0000    0.0194        38

    accuracy                         0.0098      3879
   macro avg     0.0049    0.5000    0.0097      3879
weighted avg     0.0001    0.0098    0.0002      3879

AUROC: 0.4580392989764178
Thresh=0.10 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.15 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.20 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.25 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.30 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.35 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.40 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.45 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.50 | Recall=1.0000 | Precision=0.0098 | F1=0.0194
Thresh=0.55 | Recall=0.0000 | Precision=0.0000 | 

/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/panfs/ccds02/app/modules/jupyter/ilab/pytorch-kernel/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this beh

KeyboardInterrupt: 